# 見積もり・データは一設計関連

---

## 論物変換

TODO:

## Buffer Layout

TODO:

## DMA Scheme

TODO:

---

## 誤り訂正

悩んだので ECC 内臓の上位モデルを参考に設計する

- w/o ECC (こちらを使用している) [KIOXIA_TC58NVG0S3HTA00](https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/506/KIOXIA_TC58NVG0S3HTA00_Rev2.00_E191001C.pdf)
- w/ ECC [KIOXIA_TC58BVG0S3HTA00](https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/506/KIOXIA_TC58BVG0S3HTA00_REV2.00_E20191001C.pdf)

### Datasheet確認

#### ECC & Sector definition for ECC

- 2KByte pageは 512byte * 4area + 16byte * 4area で使用している
- main + spare の 512+16=528byteごとに、 9bitの検出と8bitの訂正能力を備える
- ECC parity code は 2112-2175 (64byte) に格納するので、ここにはアクセスできないという注意書き

main+spareごとECCのペアになっている点、ECC Status Readの戻り値がArea事に分かれていること、最後のECC parity code格納から以下のように見える

| sector | main [byte] | spare [byte] | ECC parity [byte] |
| ------ | ----------- | ------------ | ----------------- |
| 1st    | 0-511       | 2048 - 2063  | 2112 - 2127       |
| 2nd    | 512 - 1023  | 2064 - 2079  | 2128 - 2143       |
| 3rd    | 1024 - 1535 | 2080 - 2095  | 2144 - 2159       |
| 4th    | 1536 - 2047 | 2096 - 2111  | 2160 - 2175       |

data = 528byte (4224bit), parity = 16byte (128bit) で同じ訂正を組むことにする。
ただ、気になるのが `9bitの検出と8bitの訂正能力を備える` とあるが、 一般(?)にいうECCは、1bitの訂正と2bitの検出が限界だったように思う。
ECCではない独自方式なのか、もしくは最大8bit検出というのはmain/spareを更に8分割したうえで、それぞれ独立した領域でエラーが出たときの理論的な条件でのCorrectableなのか。

以下を読むとCypress社のNOR Flashでは32byteごとにECC Encodeを施していて、1bit訂正2bit検出を拡張ハミング符号で実装している文言がある。総括部分の保証が若干謎の言い回しな気もするが...。

[SECDEC ECCスキームが2ビットを超えるエラーを処理する方法 - KBA222012 - Community Translated (JA) - infineaon Developer Community](https://community.infineon.com/t5/%E3%83%8A%E3%83%AC%E3%83%83%E3%82%B8%E3%83%99%E3%83%BC%E3%82%B9%E3%82%A2%E3%83%BC%E3%83%86%E3%82%A3%E3%82%AF%E3%83%AB-KBA/SECDEC-ECC%E3%82%B9%E3%82%AD%E3%83%BC%E3%83%A0%E3%81%8C2%E3%83%93%E3%83%83%E3%83%88%E3%82%92%E8%B6%85%E3%81%88%E3%82%8B%E3%82%A8%E3%83%A9%E3%83%BC%E3%82%92%E5%87%A6%E7%90%86%E3%81%99%E3%82%8B%E6%96%B9%E6%B3%95-KBA222012-Community-Translated-JA/ta-p/254390)

#### (17) Reliability Guidance

- 8bit ECC for each 512byte 必要 (これは w/o ECC modelにも記載あり)
- 0 data paddingをやめろという記載もあり。DC balance取るようなエンコーディングも必要?

### DC Balance Encode + ECC Encode

In [22]:
import os
from dataclasses import dataclass


@dataclass(frozen=True)
class DcBalanceEncodeSize:
    data_bits: int
    total_bits: int

    @property
    def code_rate(self) -> float:
        return self.data_bits / self.total_bits


@dataclass(frozen=True)
class EccEncodeSize:
    parity_bits: int
    additional_bits: int
    data_bits: int

    @property
    def total_bits(self) -> int:
        return self.data_bits + self.parity_bits + self.additional_bits

    @property
    def code_rate(self) -> float:
        return self.data_bits / self.total_bits

    @classmethod
    def from_parity_bits(
        cls, parity_bits: int, additional_bits: int = 1
    ) -> "EccEncodeSize":
        """
        Create an ECC frame size from the number of parity bits and additional bits.
        :param parity_bits: The number of parity bits.
        :param additional_bits: The number of additional bits.
        :return: The ECC frame size.
        """
        data_bits = 2**parity_bits - parity_bits - 1
        return cls(parity_bits, additional_bits, data_bits)


# USB Hostからのデータ最小単位
logical_data_bytes = 512
# 1ページあたりにUSB Hostからのデータを何個格納するか
logica_data_num_per_page = 4
# 1ページあたりのデータ量
logical_data_bytes_per_page = logical_data_bytes * logica_data_num_per_page
# 1pageあたりのデータ量
bytes_per_page = 2048 + 128
# USB Host以外に使用できる冗長データ量
redundant_bytes = bytes_per_page - logical_data_bytes_per_page

############################################
# DC Balance Encode設定ごとに見積もりを行う
dc_balance_encode_size_list = [
    DcBalanceEncodeSize(8, 10),
    DcBalanceEncodeSize(64, 66),
    DcBalanceEncodeSize(64, 67),
    DcBalanceEncodeSize(128, 130),
    DcBalanceEncodeSize(1, 1),  # disable
]
require_bytes_dc_balance_list = [
    (
        dc_balance_encode_size,
        logical_data_bytes_per_page * (1.0 / dc_balance_encode_size.code_rate),
    )
    for dc_balance_encode_size in dc_balance_encode_size_list
]
print(f"dc_balance_encode_size_list={dc_balance_encode_size_list}")
print(f"require_bytes_dc_balance_list={require_bytes_dc_balance_list}")

############################################
# ECCの設定ごとに見積もりを行う

# 計算対象にするparity長
parity_len_n = 16
ecc_encode_size_list = [
    EccEncodeSize.from_parity_bits(i) for i in range(2, parity_len_n)
]
print(f"ecc_encode_size_list={ecc_encode_size_list}")
print(f"total_bits={[encode_size.total_bits for encode_size in ecc_encode_size_list]}")
print(f"code_rate={[encode_size.code_rate for encode_size in ecc_encode_size_list]}")

# 1ページあたりのデータ量に対して、冗長データ量を割り当てたときの、合計のデータ量
require_bytes_per_page = [
    (
        ecc_encode_size,
        dc_balance_encode_size,
        require_bytes_per_page * (1.0 / ecc_encode_size.code_rate),
    )
    for ecc_encode_size in ecc_encode_size_list
    for dc_balance_encode_size, require_bytes_per_page in require_bytes_dc_balance_list
]
# 冗長データ量が足りないものは除外
ecc_encode_size_list = [
    (ecc_encode_size, dc_balance_encode_size, require_bytes)
    for ecc_encode_size, dc_balance_encode_size, require_bytes in require_bytes_per_page
    if require_bytes <= bytes_per_page
]
print(f"require_bytes_per_page={require_bytes_per_page}")
print("\n\navailable dc balance / ecc encode size")
for ecc_encode_size, dc_balance_encode_size, require_bytes in ecc_encode_size_list:
    print(f"ecc_encode_size={ecc_encode_size}, dc_balance_encode_size={dc_balance_encode_size}, require_bytes={require_bytes}, spare_bytes={bytes_per_page - require_bytes}")

dc_balance_encode_size_list=[DcBalanceEncodeSize(data_bits=8, total_bits=10), DcBalanceEncodeSize(data_bits=64, total_bits=66), DcBalanceEncodeSize(data_bits=64, total_bits=67), DcBalanceEncodeSize(data_bits=128, total_bits=130), DcBalanceEncodeSize(data_bits=1, total_bits=1)]
require_bytes_dc_balance_list=[(DcBalanceEncodeSize(data_bits=8, total_bits=10), 2560.0), (DcBalanceEncodeSize(data_bits=64, total_bits=66), 2112.0), (DcBalanceEncodeSize(data_bits=64, total_bits=67), 2144.0), (DcBalanceEncodeSize(data_bits=128, total_bits=130), 2080.0), (DcBalanceEncodeSize(data_bits=1, total_bits=1), 2048.0)]
ecc_encode_size_list=[EccEncodeSize(parity_bits=2, additional_bits=1, data_bits=1), EccEncodeSize(parity_bits=3, additional_bits=1, data_bits=4), EccEncodeSize(parity_bits=4, additional_bits=1, data_bits=11), EccEncodeSize(parity_bits=5, additional_bits=1, data_bits=26), EccEncodeSize(parity_bits=6, additional_bits=1, data_bits=57), EccEncodeSize(parity_bits=7, additional_bits=1, data_bits